<a href="https://colab.research.google.com/github/lakshmipriyaanand/LakshmiPriyaAnand/blob/main/Autocode%20Med%20-%20LLM%20powered%20medical%20code%20generation%20system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1"
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need to add extra instructions

In [ ]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [ ]:
from unsloth import FastModel
import torch
max_seq_length = 1024

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 11-24 11:14:19 [__init__.py:244] Automatically detected platform cuda.
ERROR 11-24 11:14:22 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Gemma3 patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.
Unsloth: QLoRA and full finetuning al

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
## Importing dataset
from datasets import load_dataset
#dataset = load_dataset("ayrus08/medicalcoding-synthetic", split = "train")
dataset = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k", split = "train")

dataset


README.md:   0%|          | 0.00/542 [00:00<?, ?B/s]

data/train-00000-of-00001-5e7cb295b9cff0(…):   0%|          | 0.00/70.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 112165
})

In [ ]:
#dataset[0]["summary"]
#dataset[0]["input"]
#dataset[0]["codes description"]
dataset[0]["output"]


In [ ]:
def extract_hash_answer(text):
    if "\n" not in text: return None
    return text.split("\n")[1].strip()
extract_hash_answer(dataset[0]["output"])

In [ ]:
complain_start = "<start_working_out>"
complain_end   = "<end_working_out>"
code_start = "<root_cause>"
code_end = "</root_cause>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {complain_start} and {complain_end}.
Then, provide your solution between {code_start}{code_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <root_cause></root_cause>'

In [ ]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["input"]},
    ],
    "answer": extract_hash_answer(x["output"]),
})
dataset[0]

Map:   0%|          | 0/112165 [00:00<?, ? examples/s]

{'instruction': "If you are a doctor, please answer the medical questions based on the patient's description.",
 'input': 'I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!',
 'output': 'Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral vertigo. In this condition, the most common symptom is dizziness or giddiness, which is made worse with movements. Accompanying naus

In [ ]:
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"\
    rf"{complain_start}.+?{complain_end}.*?"\
    rf"{code_start}(.+?){code_end}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

In [ ]:
match_format.search(
    "<start_working_out>Let me think!<end_working_out>"\
    "<root_cause>2</root_cause>",
)

In [ ]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

In [ ]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!
        score += 0.5 if response.count(complain_start) == 1 else -0.5
        score += 0.5 if response.count(complain_end)   == 1 else -0.5
        score += 0.5 if response.count(code_start)  == 1 else -0.5
        score += 0.5 if response.count(code_end)    == 1 else -0.5
        scores.append(score)
    return scores

In [ ]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(0)
            continue
        # Correct answer gets 3 points!
        if guess == true_answer:
            score += 3.0
        # Match if spaces are seen
        elif (str(guess).strip() == str(true_answer).strip()):
            score += 1.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 0.5
                elif ratio >= 0.8 and ratio <= 1.2: score += 0.25
                else: score -= 1.0 # Penalize wrong answers
            except:
                score -= 0.5 # Penalize
        scores.append(score)
    return scores

In [ ]:
match_numbers = re.compile(
    rf"{complain_start}.*?([\d\.]{{1,}})",
    flags = re.MULTILINE | re.DOTALL
)
match_numbers.findall("<root_cause>  0.34  </root_cause>")

[]

In [ ]:
def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    print('*'*20, f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(0)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            guess       = float(guess.strip())
            scores.append(1.5 if guess == true_answer else 0.0)
        except:
            scores.append(0)
            continue
    return scores

In [ ]:
max_prompt_length = 256

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 50,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

Unsloth: Switching to float32 training since model cannot work with float16


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 112,165 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 6,522,880 of 1,006,408,832 (0.65% trained)
`generation_config` default values have been modified to match model-specific defaults: {'max_length': 32768, 'top_p': 0.95}. If this is not desired, please set these values explicitly.


******************** Question:
I m extremely itchy all over my body, the itch happens mostly at night, and in my room. I have washed my sheets and and now even on a sheet-less air mattress. The itch occurs at all points of my day except when I am in the shower. Baby powder and cortisone doesn t help at all. The itch is mostly on my chest, thighs, arms, and hands (and avoids the pubic area). PLEASE HELP I don t have any sort of bites; but I have noticed that much of the itchy areas house small red dots, and when I get out of the shower, some areas of my skin get very red (but are relieved). 
Answer:
None 
Response:
<start_working_out>
The presented symptoms – intense, night-time itching all over the body, primarily in the chest, thighs, arms, and hands, with avoidance of the pubic area – are highly suggestive of a **hive eruption** or **contact dermatitis**.  The combination of the night-time onset, localized distribution, avoidance of the pubic area, and the observed red spots after sh

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000000,0.750000,0.500000,593.000000,268.000000,730.000000,0.000000,593.000000,268.000000,730.000000,0.000000,0.000000,0.000000,0.750000,0.500000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.250000,0.500000,288.500000,204.000000,380.000000,0.000000,288.500000,204.000000,380.000000,0.000000,0.000000,0.000000,0.250000,0.500000,0.000000,0.000000,0.000000,0.000000
3,0.000000,1.000000,0.000000,559.500000,343.000000,713.000000,0.000000,559.500000,343.000000,713.000000,0.000012,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,1.250000,0.957427,458.500000,196.000000,711.000000,0.000000,458.500000,196.000000,711.000000,0.000024,0.000000,0.000000,1.250000,0.957427,0.000000,0.000000,0.000000,0.000000
5,0.000000,1.750000,0.500000,576.250000,384.000000,768.000000,0.250000,512.333374,384.000000,600.000000,0.000014,0.000000,0.000000,1.750000,0.500000,0.000000,0.000000,0.000000,0.000000
6,0.000000,1.750000,0.500000,688.500000,575.000000,768.000000,0.500000,609.000000,575.000000,643.000000,0.000015,0.000000,0.000000,1.750000,0.500000,0.000000,0.000000,0.000000,0.000000
7,0.000000,1.000000,0.816497,302.250000,147.000000,572.000000,0.000000,302.250000,147.000000,572.000000,0.000087,0.000000,0.000000,1.000000,0.816497,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.750000,0.500000,544.250000,272.000000,768.000000,0.500000,320.500000,272.000000,369.000000,0.000041,0.000000,0.000000,0.750000,0.500000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.750000,0.500000,239.250000,102.000000,325.000000,0.000000,239.250000,102.000000,325.000000,0.000528,0.000000,0.000000,0.750000,0.500000,0.000000,0.000000,0.000000,0.000000
10,0.000000,-2.000000,0.000000,768.000000,768.000000,768.000000,1.000000,0.000000,0.000000,0.000000,0.000010,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Unsloth: Will smartly offload gradients to save VRAM!
******************** Question:
I am constantly  giddy , with a swinging feeling and have to fight  against the falling feeling..Sometimes feeling  nauseated.   I am nursing my husband 24hr   with help only to shower him...When I get up to him at night I feel dreadful and manage to land on the bed..Takes  few minutes to be able to tend to him...I get very angry about this hindrance,  which probably makes it worse. 
Answer:
None 
Response:
<start_working_out>
This presents a complex scenario with potential underlying issues. The patient’s experience of “giddy” feeling, swinging sensations, and a struggle against falling sensations, coupled with nausea, suggests a potentially significant neurological or psychological component. The isolation of caring for a partner 24/7, combined with limited help and a feeling of being hindered, is likely contributing to distress. The reported anger stemming from the perceived difficulty is also a key

In [ ]:
essages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": " wisdom tooth extracted nearly 3 weeks ago. unfortunately some of the block went into my jaw muscle and I now have trismus. More alarming though is that my facial muscles feel very much weekened and smiling is an effort. Also I have had twitching above my lip and up cheek and under eye.  please help"},

    ]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20000, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": " just got bitten to my year old dog 10 min ago. this morning he was seeming like he was panting and panting and breathing very heavily. We gave him water but he still did it. Now the bite feels tingily and its like its beating.I already put poroxide on. Do you know if this is normal or what to do?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20000, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content":  "ifenac mr tablet was taken by my wife age 29 due to the pain in her lower waist , the reason is i suppose body cramp due to a massage by a proffesional.now she is suffering a severe pain in her waist and could not able to move and walk. should i go for a injection,pain killer or any spray pain relief ?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 20000, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

In [ ]:
model.save_pretrained("gemma-3")  # saving locally
tokenizer.save_pretrained("gemma-3")